In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

from geopy.geocoders import Nominatim


In [2]:
data = pd.read_csv('_src\AccidentesAviones.csv')

In [3]:
# primer vistazo
data.sample(10)

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
2827,2827,"November 18, 1975",?,"Caoba, Guatemala",Aviateca,?,Uaxactun - Flores,Douglas DC-3,TG-AGA,6142,22,18,4,15,11,4,0,Crashed into mountainous jungle in poor visibi...
4103,4103,"August 07, 1997",1236,"Miami, Florida",Fine Air,101A,Miami - Santo Domingo,McDonnell Douglas DC-8-61F,N27UA,45942/349,4,1,3,4,1,3,1,Immediately after taking off the plane went in...
1554,1554,"January 12, 1956",0340,"Frederick Henry Bay, Australia",Ansett ANA,?,Melbourne - Hobart,Douglas C-47,VH-BZA,4651,2,0,2,1,0,1,0,The cargo plane flew into the water upon appro...
4477,4477,"November 08, 2003",1026,"200 miles NE of Derby, Australia",Heliwork WA Pty Ltd. - Air Taxi,?,?,Robertson R44 helicopter,VH-YKL,0170,4,3,1,4,3,1,0,The helicopter crashed in the remote outback a...
1951,1951,"March 27, 1962",1745,"Santiago de Cuba, Cuba",Cubana de Aviacion,?,Santiago de Cuba - Havana,Ilyushin IL-14,CU-T819,?,22,18,4,22,18,4,0,Crashed shortly after takeoff into the sea 1 m...
2928,2928,"September 04, 1977",?,"Near Cuenca, Ecuador",Servicios Aereos Nacionales Airways,?,Guayaquil - Cuenca,Vickers Viscount 764D,HC-BCL,185,33,29,4,33,29,4,0,Struck a mountain peak in the Cajas mountains ...
2236,2236,"August 11, 1966",?,"Near Sibiu, Romania",Tarom,?,Bra?ov - Constanta,Ilyushin IL-14,YR-TAN,18428005,24,20,4,24,20,4,0,Crashed on a flight while en route.
310,310,"April 03, 1933",?,"Hemkenrode, Germany",Deutsche Lufthansa,?,?,Junkers F-13,D-534,?,2,1,1,1,1,0,0,Crashed under unknown circumstances.
4378,4378,"February 15, 2002",0608,"Monrovia, Liberia",Tiramavia,?,?,Antonov 12BP,ER-ADL,4342610,10,3,7,1,0,1,0,The cargo plane crashed while attempting to ma...
3607,3607,"November 25, 1989",0748,"Seoul, South Korea",Korean Airlines,?,Seou - Ulsan,Fokker F-28 Fellowship 4000,HL-7255,11223,52,46,6,1,1,0,0,The plane crashed during takeoff after losing ...


In [4]:
# veo que está completo, pero todos los campos como 'object', no hay datos numéricos 
# excepto la 1er columna que es un indice repetido
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              5008 non-null   int64 
 1   fecha                   5008 non-null   object
 2   HORA declarada          5008 non-null   object
 3   Ruta                    5008 non-null   object
 4   OperadOR                5008 non-null   object
 5   flight_no               5008 non-null   object
 6   route                   5008 non-null   object
 7   ac_type                 5008 non-null   object
 8   registration            5008 non-null   object
 9   cn_ln                   5008 non-null   object
 10  all_aboard              5008 non-null   object
 11  PASAJEROS A BORDO       5008 non-null   object
 12  crew_aboard             5008 non-null   object
 13  cantidad de fallecidos  5008 non-null   object
 14  passenger_fatalities    5008 non-null   object
 15  crew

Columnas a eliminar:

In [5]:
# elimino columna indice duplicado
data.drop('Unnamed: 0', axis=1, inplace=True) 


In [6]:
# quito el horario, no lo voy a usar para el análisis
data.drop('HORA declarada', axis=1, inplace=True)

In [7]:
# quito el tipo de fuselaje
data.drop('cn_ln', axis=1, inplace=True)

Recorro columnas:

General (aplicar a todas las columnas):

In [8]:
# veo que los faltantes estan como el string: '?'
# reemplazo por NaN
data.replace('?', 'NaN', inplace=True)

Columna Fecha

In [9]:
 #convierto la fecha a formato fecha
data['fecha'] = pd.to_datetime(data['fecha'])

In [10]:
data['fecha'].describe()

C:\Users\FedeGP\AppData\Local\Temp\ipykernel_13724\847207859.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data['fecha'].describe()


count                    5008
unique                   4577
top       1988-08-31 00:00:00
freq                        4
first     1908-09-17 00:00:00
last      2021-07-06 00:00:00
Name: fecha, dtype: object

### Columnas numéricas:
- 'all_aboard': humanos a bordo (todos) pasajeros + crew
- 'PASAJEROS A BORDO': pasajeros a bordo
- 'crew_aboard': tripulacion a bordo
- 'cantidad de fallecidos': cant de fallecidos total (totales) pasajeros + crew
- 'passenger_fatalities': pasajeros fallecidos
- 'crew_fatalities': tripulacion fallecidos
- 'ground': muertos en tierra a causa del accidente

In [11]:
# las cambio a tipo de dato numérico
data['all_aboard'] = data['all_aboard'].astype(float)
data['PASAJEROS A BORDO'] = data['PASAJEROS A BORDO'].astype(float)
data['crew_aboard']=data['crew_aboard'].astype(float)
data['cantidad de fallecidos']=data['cantidad de fallecidos'].astype(float)
data['passenger_fatalities']=data['passenger_fatalities'].astype(float)
data['crew_fatalities']=data['crew_fatalities'].astype(float)
data['ground']=data['ground'].astype(float)

columna 'cantidad de fallecidos'

In [12]:
# elimino los registros que no tienen ningun dato de muerto, ya que no serviran para nada en el análisis
# no tienen los datos de las otras columnas tampoco
data.drop(data[data['cantidad de fallecidos'].isnull()].index, inplace=True)

Columna: 'all_aboard'

In [13]:
# genero columna auxiliar con el máximo entre: all_aboard, todos los fallecidos, todos los a bordo sumando (pass+crew) y todos los fallecidos sumando (pass+crew)
# teniendo en cuenta los NaN como 0
data['pas+crew']=np.nansum(data[['PASAJEROS A BORDO','crew_aboard']], axis=1)
data['pass+crew_fat']= np.nansum(data[['passenger_fatalities','crew_fatalities']], axis=1)
data['maximo'] = np.nanmax(
    data[['all_aboard','cantidad de fallecidos','pas+crew','pass+crew_fat']], axis=1)

# Ese maximo sera mi nuevo all_board
data['all_aboard']=data['maximo']
# elimino las columnas auxiliares
data.drop(['maximo','pas+crew','pass+crew_fat'], axis=1, inplace=True)

Columna: 'crew_aboard'

In [14]:
#obj: llenar nulos de crew_aboard con el valor mínimo entre la media de crew_aboard, y el all_aboard
# creo col auxiliar con el valores de la media de crew_aboard.
data['aux']= round(data['crew_aboard'].mean(),0)
# creo col auxiliar con el minimo entre la anterior y all_aboard
data['minimo']=np.min(data[['all_aboard','aux']], axis=1)
# imputo a faltantes
data['crew_aboard'].fillna(data['minimo'], inplace=True)


Columna 'PASAJEROS A BORDO'

In [15]:
# imputo a los faltantes la diferencia entre all_aboard y crew_aboard
data['PASAJEROS A BORDO']= data['all_aboard']-data['crew_aboard']
# borro las col auxiliares
data.drop(['aux','minimo'], axis=1, inplace=True)


Columna: 'passenger_fatalities' y 'crew_fatalities'

In [16]:
# obj: asignar 'cantidad de fallecidos' a esas columnas repartiendo proporcionalmente a cómo estaba abordado el avion según
# ('crew_aboard' y 'pasajeros a bordo')

# estimo cantidad de fallecidos de la crew, en una nueva columna
data['estim_fat_crew']=round(data['crew_aboard']/data['all_aboard']*data['cantidad de fallecidos'],0)
#lleno crew fatalities con el minimo entre la estimacion y la cantidad total de muertos (por si en el redondeo se agrando el numero)
data['crew_fatalities'].fillna(np.min(data[['estim_fat_crew','cantidad de fallecidos']], axis=1), inplace=True)
# agrego columnas auxiliar multiplicandola por -1, para luego efectuar una resta al sumar las columnas
data['neg_crew_fatalities']=data['crew_fatalities']*-1
data['passenger_fatalities'].fillna(np.sum(data[['cantidad de fallecidos','neg_crew_fatalities']], axis=1), inplace=True)

# elimino columnas auxiliares
data.drop(['estim_fat_crew','neg_crew_fatalities'], axis=1, inplace=True)

Columna 'ground'

In [17]:
# En columna ground si no hay dato, imputo 0
data['ground'].fillna(0, inplace=True)

In [18]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
all_aboard,5000.0,31.1544,45.548259,0.0,7.0,16.0,35.0,644.0
PASAJEROS A BORDO,5000.0,26.6240,43.580428,0.0,3.0,12.0,30.0,614.0
crew_aboard,5000.0,4.5304,3.682144,0.0,2.0,4.0,6.0,83.0
cantidad de fallecidos,5000.0,22.2940,35.000385,0.0,4.0,11.0,25.0,583.0
passenger_fatalities,5000.0,18.7546,33.447453,0.0,1.0,8.0,21.0,560.0
crew_fatalities,5000.0,3.6082,3.123755,0.0,2.0,3.0,5.0,43.0
ground,5000.0,1.7060,55.295550,0.0,0.0,0.0,0.0,2750.0


### Columnas categóricas

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 5007
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fecha                   5000 non-null   datetime64[ns]
 1   Ruta                    5000 non-null   object        
 2   OperadOR                5000 non-null   object        
 3   flight_no               5000 non-null   object        
 4   route                   5000 non-null   object        
 5   ac_type                 5000 non-null   object        
 6   registration            5000 non-null   object        
 7   all_aboard              5000 non-null   float64       
 8   PASAJEROS A BORDO       5000 non-null   float64       
 9   crew_aboard             5000 non-null   float64       
 10  cantidad de fallecidos  5000 non-null   float64       
 11  passenger_fatalities    5000 non-null   float64       
 12  crew_fatalities         5000 non-null   float64 

Columna 'Ruta' (lugar del accidente); y adicionar columnas de country donde ocurrio el accidente y (lat, long)

In [20]:
data['Ruta']

0                      Fort Myer, Virginia
1                  Juvisy-sur-Orge, France
2                Atlantic City, New Jersey
3       Victoria, British Columbia, Canada
4                       Over the North Sea
                       ...                
5003                    Near Butte, Alaska
5004                  Near Kaduna, Nigeria
5005            Near Pyin Oo Lwin, Myanmar
5006            Patikul, Sulu, Philippines
5007                        Palana, Russia
Name: Ruta, Length: 5000, dtype: object

In [21]:

# normalizamos Ruta, para poder detectar la ubicación
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Over the ', ''))
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Near ',''))
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Off ',''))
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('North of ',''))
data['Ruta']=data['Ruta'].apply(lambda x: x.replace(' Bay,',','))
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('La Rache, Morocco', 'Larache, Morocco')) #233
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Territory of New Guinea', 'New Guinea')) #1038
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Santiago del Estado','Santiago del Estero')) #1225
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Alvarado, California','Union City, California')) #1382
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Castell de Fels','Castelldefels')) #1692
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Santa Maria, California','Santa Barbara, California')) #1791
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Barnegat City','Barnegat')) #1837
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('300 nm NW of ','')) #3320
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Berlin, East Germany','Berlin, Germany')) #3412
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Nasevou Village','Nasevo St')) #4221
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Ampisibil Village', 'Oksibil')) #4738
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Pavillon', 'Genesee')) #413
data['Ruta']=data['Ruta'].apply(lambda x: x.replace('Desertores Island Región de Los Lagos', 'Desertores Island, Chile')) #464

'''
Toda esta sección fue un intento de describir con más precisión la ubicación, perpo fue imposible
Excesivamente lento, y totalmente impredecible la búsqueda.
Arroja muchos NaN, a veces encuentra bien, a veces da error.

geolocator = Nominatim(user_agent='hola')
data.loc[:500,'coord_Ruta'] = data.loc[:500,'Ruta'].apply(geolocator.geocode)
data.loc[501:1000,'coord_Ruta'] = data.loc[501:1000,'Ruta'].apply(geolocator.geocode)
data.loc[1001:1500,'coord_Ruta'] = data.loc[1001:1500,'Ruta'].apply(geolocator.geocode)
data.loc[1501:2000,'coord_Ruta'] = data.loc[1501:2000,'Ruta'].apply(geolocator.geocode)
data.loc[2001:2500,'coord_Ruta'] = data.loc[2001:2500,'Ruta'].apply(geolocator.geocode)
data.loc[2501:3000,'coord_Ruta'] = data.loc[2501:3000,'Ruta'].apply(geolocator.geocode)
data.loc[3001:3500,'coord_Ruta'] = data.loc[3001:3500,'Ruta'].apply(geolocator.geocode)
data.loc[3501:4000,'coord_Ruta'] = data.loc[3501:4000,'Ruta'].apply(geolocator.geocode)
data.loc[4001:4500,'coord_Ruta'] = data.loc[4001:4500,'Ruta'].apply(geolocator.geocode)
data.loc[4501:,'coord_Ruta'] = data.loc[4501:,'Ruta'].apply(geolocator.geocode)
'''

"\nToda esta sección fue un intento de describir con más precisión la ubicación, perpo fue imposible\nExcesivamente lento, y totalmente impredecible la búsqueda.\nArroja muchos NaN, a veces encuentra bien, a veces da error.\n\ngeolocator = Nominatim(user_agent='hola')\ndata.loc[:500,'coord_Ruta'] = data.loc[:500,'Ruta'].apply(geolocator.geocode)\ndata.loc[501:1000,'coord_Ruta'] = data.loc[501:1000,'Ruta'].apply(geolocator.geocode)\ndata.loc[1001:1500,'coord_Ruta'] = data.loc[1001:1500,'Ruta'].apply(geolocator.geocode)\ndata.loc[1501:2000,'coord_Ruta'] = data.loc[1501:2000,'Ruta'].apply(geolocator.geocode)\ndata.loc[2001:2500,'coord_Ruta'] = data.loc[2001:2500,'Ruta'].apply(geolocator.geocode)\ndata.loc[2501:3000,'coord_Ruta'] = data.loc[2501:3000,'Ruta'].apply(geolocator.geocode)\ndata.loc[3001:3500,'coord_Ruta'] = data.loc[3001:3500,'Ruta'].apply(geolocator.geocode)\ndata.loc[3501:4000,'coord_Ruta'] = data.loc[3501:4000,'Ruta'].apply(geolocator.geocode)\ndata.loc[4001:4500,'coord_Ruta

In [22]:
data['localizar'] = data['Ruta'].apply(lambda x: x.split(','))
data['localizar'] = data['localizar'].apply(lambda x: x[-1])
data['localizar'] = data['localizar'].apply(lambda x: x.strip())


paises = pd.DataFrame()
paises['localizar'] = data['localizar'].unique()
paises['pais'] = paises['localizar']
paises['pais'] = paises['pais'].apply(lambda x: x.replace('East Sardinia', 'Sardinia, Italy'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('110 miles West of Ireland', 'Ireland'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Unied Kingdom', 'United Kingdom'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Californiia', 'California'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('French West Africa', 'Senegal'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('U.S. Samoa', 'Samoa'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Cailifornia', 'California'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Pacific Ocean between Manila and Guam', 'Manila'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Atlantic Ocean between N.Y. and Bermuda', 'New York'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Coatia', 'Croatia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Australila', 'Australia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Dutch Guyana', 'Suriname'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Swden', 'Sweden'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Jacquinot Bay  New Guinea', 'New Guinea'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('French Equatorial Africa', 'Republic of the Congo'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('the Panama coast', 'Panama'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Gulf of Karkinitsky', 'Ukraine'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('French Cameroons', 'Cameroon'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('800 miles east of Newfoundland', 'Newfoundland and Labrador'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('French Indo-China', 'Vietnam'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Pacific Ocean between Hong Kong and Macao', 'Hong Kong'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('off Bermuda', 'Bermuda'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('off the Philippine island of Elalat', 'Philippines'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Atlantic Ocean off Florida', 'Miami, Florida'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('British Cameroons', 'Cameroon'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Bugaria', 'Bulgaria'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('325 miles east of Wake Island', 'Wake Island'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('South Dekota', 'South Dakota'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Baangladesh', 'Bangladesh'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Aregntina', 'Argentina'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Midway Island Naval Air Station', 'Hawaii'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Massachutes', 'Massachusetts'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Mt. Helmos. Greece', 'Greece'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('French Somaliland', 'Somalia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('South-West  Africa (Namibia)', 'Namibia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Alakska', 'Alaska'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Khmer Republic', 'Cambodia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Southeastern Bolivia', 'Bolivia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Rhodesia (Zimbabwe)', 'Zimbabwe'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Djbouti', 'Djibouti'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Centeral Afghanistan\rAfghanistan', 'Afghanistan'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Sierre Leone', 'Sierra Leone'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Airzona', 'Arizona'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('En route from Argentina  to  California', 'Argentina'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Inodnesia', 'Indonesia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Domincan Republic', 'Dominican Republic'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Canada2', 'Canada'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Yugosalvia', 'Serbia'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('off Angola', 'Angola'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Calilfornia', 'California'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Sao Gabriel de Cachoeria', 'Amazonas, Brazil'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Democtratic Republic Congo', 'Congo'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Democratic Republic Cogo', 'Congo'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('175 miles off the Egyptian coast', 'Egypt'))
paises['pais'] = paises['pais'].apply(lambda x: x.replace('Minnisota','Minnesota'))


In [23]:

geolocator = Nominatim(user_agent='hola')
# paises['coord'] = paises['pais'].apply(geolocator.geocode)
paises.loc[:100,'coord'] = paises.loc[:100,'pais'].apply(geolocator.geocode)
# 5 min aprox.
paises.loc[101:200,'coord'] = paises.loc[101:200,'pais'].apply(geolocator.geocode)
paises.loc[201:300,'coord'] = paises.loc[201:300,'pais'].apply(geolocator.geocode)
paises.loc[301:400,'coord'] = paises.loc[301:400,'pais'].apply(geolocator.geocode)
paises.loc[401:,'coord'] = paises.loc[401:,'pais'].apply(geolocator.geocode)
# 4min aprox

In [24]:
paises

,localizar,pais,coord
0,Virginia,Virginia,"(Virginia, United States, (37.1232245, -78.492..."
1,France,France,"(France, (46.603354, 1.8883335))"
2,New Jersey,New Jersey,"(New Jersey, United States, (40.0757384, -74.4..."
3,Canada,Canada,"(Canada, (61.0666922, -107.991707))"
4,North Sea,North Sea,(North Sea / Nordsee / Noordzee / Nordsøen / N...
...,...,...,...
493,Malta International Airport,Malta International Airport,"(Ajruport Internazzjonali ta' Malta, Triq De H..."
494,South Australia,South Australia,"(South Australia, Australia, (-30.5343665, 135..."
495,Mississipi,Mississipi,"(Mississippi, United States, (32.9715285, -89...."
496,Guizhou Province,Guizhou Province,"(贵洲, 开平市, 江门市, 广东省, 中国, (22.3175945, 112.60320..."


In [25]:
paises['lat']=paises['coord'].apply(lambda x: x.latitude)

In [26]:
paises['lon']=paises['coord'].apply(lambda x: x.longitude)

In [27]:
paises['lat-lon']=paises['lat'].astype(str)+', '+paises['lon'].astype(str)

In [28]:
paises['country']=paises['lat-lon'].apply(lambda x: geolocator.reverse(x, language='en'))
# 4 min aprox

In [29]:
paises[paises['country'].isnull()]

,localizar,pais,coord,lat,lon,lat-lon,country
4,North Sea,North Sea,(North Sea / Nordsee / Noordzee / Nordsøen / N...,56.741234,2.891172,"56.74123355, 2.8911717166880475",None
30,North Atlantic Ocean,North Atlantic Ocean,"(北大西洋, (40.0, -40.0))",40.000000,-40.000000,"40.0, -40.0",None
36,Mediterranean Sea,Mediterranean Sea,"(Mediterranean Sea, (35.0000035, 19.9999957))",35.000003,19.999996,"35.0000035, 19.9999957",None
111,South Atlantic Ocean,South Atlantic Ocean,"(Južni Atlantik, (-45.0, -15.0))",-45.000000,-15.000000,"-45.0, -15.0",None
145,Gulf of Oman,Gulf of Oman,"(خلیج عمان, (24.213802899999997, 58.9845822868...",24.213803,58.984582,"24.213802899999997, 58.9845822868196",None
150,Sea of Japan,Sea of Japan,"(Японское море / 日本海 / 동해, (43.060053350000004...",43.060053,137.335301,"43.060053350000004, 137.33530126992395",None
157,Indian Ocean,Indian Ocean,"(Indian Ocean, (-9.9999999, 69.9999999))",-10.000000,70.000000,"-9.9999999, 69.9999999",None
170,Pacific Ocean,Pacific Ocean,"(Pacific Ocean, (-0.703107, -120.9375))",-0.703107,-120.937500,"-0.703107, -120.9375",None
194,North Pacific Ocean,North Pacific Ocean,"(North Pacific Ocean, (30.0, -170.0))",30.000000,-170.000000,"30.0, -170.0",None
204,Persian Gulf,Persian Gulf,"(الخليج العربي, (27.2380913, 51.19723479885998))",27.238091,51.197235,"27.2380913, 51.19723479885998",None


In [30]:
def entregar_pais(x):
    try:
        a = x.raw['address']['country']
        return a
    except:
        return 'Water'

In [31]:
paises['country2']=paises['country'].apply(entregar_pais)

In [32]:
paises

,localizar,pais,coord,lat,lon,lat-lon,country,country2
0,Virginia,Virginia,"(Virginia, United States, (37.1232245, -78.492...",37.123224,-78.492772,"37.1232245, -78.4927721","(Prince Edward County, Virginia, United States...",United States
1,France,France,"(France, (46.603354, 1.8883335))",46.603354,1.888334,"46.603354, 1.8883335","(D 51, Tranzault, La Châtre, Indre, Centre-Val...",France
2,New Jersey,New Jersey,"(New Jersey, United States, (40.0757384, -74.4...",40.075738,-74.404162,"40.0757384, -74.4041622","(Woodair Estates, Jackson Township, Ocean Coun...",United States
3,Canada,Canada,"(Canada, (61.0666922, -107.991707))",61.066692,-107.991707,"61.0666922, -107.991707","(South Slave Region, Northwest Territories, Ca...",Canada
4,North Sea,North Sea,(North Sea / Nordsee / Noordzee / Nordsøen / N...,56.741234,2.891172,"56.74123355, 2.8911717166880475",None,Water
...,...,...,...,...,...,...,...,...
493,Malta International Airport,Malta International Airport,"(Ajruport Internazzjonali ta' Malta, Triq De H...",35.847397,14.492295,"35.84739655, 14.492295178989536","(Malta International Airport, Triq De Hosa, Sa...",Malta
494,South Australia,South Australia,"(South Australia, Australia, (-30.5343665, 135...",-30.534367,135.630121,"-30.5343665, 135.6301212","(Bon Bon, Pastoral Unincorporated Area, South ...",Australia
495,Mississipi,Mississipi,"(Mississippi, United States, (32.9715285, -89....",32.971528,-89.734850,"32.9715285, -89.7348497","(County Road 4102, Joseph, Attala County, Miss...",United States
496,Guizhou Province,Guizhou Province,"(贵洲, 开平市, 江门市, 广东省, 中国, (22.3175945, 112.60320...",22.317594,112.603201,"22.3175945, 112.6032007","(Guizhou, Kaiping, Jiangmen, Guangdong Provinc...",China


In [33]:
place= pd.DataFrame()
place[['localizar','country', 'lat', 'lon']] = paises[['localizar', 'country2', 'lat', 'lon']]

In [34]:
data=data.merge(right=place, how='left', on='localizar', validate='m:1')

Columnas: OperadOR y flight number (nada para hacer, ni normalizar)

Columna: route

In [35]:
data['route']=data['route'].apply(lambda x: x.replace('–','-'))

In [36]:
def guion(x):
    if '-' in str(x):
        return 'Comercial'
    else:
        return 'Otro'

# en principio todos los que tienen ruta de origen y final (incluye guion) los clasifico como Comerciales, y al resto como otro

In [90]:
#Abro la columna con nulos
data['tipo_vuelo']=np.NaN

In [91]:
data['tipo_vuelo']=data['route'].apply(lambda x: guion(x))

In [92]:

def militar_route(x):
    word_list = ['mission', 'Bombing', 'Militar', 'troop']
    a=0
    for i in word_list:
        if i in x:
            a= 1
    
    if a== 1: 
        return 'Militar'
    else:
        return 'Otro'
    



In [93]:
# sobre los que preclasifiqué con Otro, me fijo si son militares, y les pongo Militar, sino dejo 'Otro'
data.loc[data[data['tipo_vuelo']=='Otro'].index,'tipo_vuelo']=data['route'].apply(lambda x: militar_route(x))

In [94]:
def militar_operador(x):
    word_list = ['Militar', 'Air Force', 'Army', 'Navy']
    a=0
    for i in word_list:
        if i in x:
            a= 1
    
    if a== 1: 
        return 'Militar'
    else:
        return 'Otro'


In [95]:
#repito lo anterior, pero fijándome en otra columna si hay info de militar
data.loc[data[data['tipo_vuelo']=='Otro'].index,'tipo_vuelo']=data['OperadOR'].apply(lambda x: militar_operador(x))

In [101]:
data[data['OperadOR'].str.contains('Militar')]

,fecha,Ruta,OperadOR,flight_no,route,ac_type,registration,all_aboard,PASAJEROS A BORDO,crew_aboard,...,crew_fatalities,ground,summary,localizar,country,lat,lon,tipo_vuelo,tipo_avion,intencional
0,1908-09-17,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,2.0,1.0,1.0,...,0.0,0.0,"During a demonstration flight, a U.S. Army fly...",Virginia,United States,37.123224,-78.492772,Militar,Avión,0
2,1912-07-12,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,5.0,0.0,5.0,...,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,New Jersey,United States,40.075738,-74.404162,Militar,Otro,0
4,1913-09-09,North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,20.0,15.0,5.0,...,4.0,0.0,The airship flew into a thunderstorm and encou...,North Sea,Water,56.741234,2.891172,Militar,Otro,0
5,1913-10-17,"Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,28.0,23.0,5.0,...,5.0,0.0,Hydrogen gas which was being vented was sucked...,Germany,Germany,51.163818,10.447831,Militar,Otro,0
6,1915-03-05,"Tienen, Belgium",Military - German Navy,NaN,NaN,Zeppelin L-8 (airship),NaN,41.0,0.0,41.0,...,17.0,0.0,Crashed into trees while attempting to land af...,Belgium,Belgium,50.640281,4.666715,Militar,Otro,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980,2019-07-30,"Rawalpindi, India",Military - Pakistan Army,NaN,Training,Beechcraft B300 King Air,766,5.0,0.0,5.0,...,5.0,14.0,"The Pakistani military plane, on a training fl...",India,India,22.351115,78.667743,Militar,Avión,0
4992,2020-09-25,"Chuguev, Ukraine",Military - Ukraine Air Force,NaN,Training,Antonov An26SH,76 yellow,27.0,20.0,7.0,...,7.0,0.0,"The military transport, crashed 1.2 miles from...",Ukraine,Ukraine,49.487197,31.271832,Militar,Avión,0
4996,2021-05-21,"Kaduna, Nigeria",Military - Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,11.0,7.0,4.0,...,4.0,0.0,"While on final approach, in poor weather condi...",Nigeria,Nigeria,9.600036,7.999972,Militar,Avión,0
4997,2021-06-10,"Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,14.0,12.0,2.0,...,1.0,0.0,The plane was carrying military personnel and ...,Myanmar,Myanmar,17.175050,95.999965,Militar,Avión,0


In [97]:
# me falta revisar bien los preclasificados como comerciales, a ver si son militares.
def militar_operador_acomerciales(x):
    word_list = ['Militar', 'Military', 'Air Force', 'Army', 'Navy']
    a=0
    for i in word_list:
        if i in x:
            a= 1
    
    if a== 1: 
        return 'Militar'
    else:
        return 'Comercial'

In [98]:
data.loc[data[data['tipo_vuelo']=='Comercial'].index,'tipo_vuelo']=data['OperadOR'].apply(lambda x: militar_operador_acomerciales(x))

In [99]:
# data[data['tipo_vuelo']=='Comercial']['PASAJEROS A BORDO'].mean()
# 31 aprox

31.224142816661942

In [100]:
# data[data['tipo_vuelo'].isnull()]

,fecha,Ruta,OperadOR,flight_no,route,ac_type,registration,all_aboard,PASAJEROS A BORDO,crew_aboard,...,crew_fatalities,ground,summary,localizar,country,lat,lon,tipo_vuelo,tipo_avion,intencional


In [68]:
# lleno nulos de la columna tipo_vuelo con 'Comercial' si la cant de pasajeros es > 31; y con 'Otro' si es menor

# data.loc[data[data['PASAJEROS A BORDO']>30|data['tipo_vuelo'].isnull()].index,'tipo_vuelo']='Comercial'

In [69]:
# data['tipo_vuelo'].fillna('Otro', inplace=True)

In [103]:
data['tipo_vuelo'].describe()

count          5000
unique            3
top       Comercial
freq           3529
Name: tipo_vuelo, dtype: object

Columna: ac_type (la uso para clasificar en: Avion u Otro)

In [104]:
data['ac_type'].unique().tolist()

['Wright Flyer III',
 'Wright Byplane',
 'Dirigible',
 'Curtiss seaplane',
 'Zeppelin L-1 (airship)',
 'Zeppelin L-2 (airship)',
 'Zeppelin L-8 (airship)',
 'Zeppelin L-10 (airship)',
 'Schutte-Lanz S-L-10 (airship)',
 'Zeppelin L-32 (airship)',
 'Zeppelin L-31 (airship)',
 'Super Zeppelin (airship)',
 'Zeppelin L-34 (airship)',
 'Airship',
 'Schutte-Lanz S-L-9 (airship)',
 'Zeppelin L-22 (airship)',
 'Zeppelin L-43 (airship)',
 'Zepplin L-48 (air ship)',
 'Zeppelin L-23 (airship)',
 'Zeppelin L-44 (airship)',
 'Zeppelin L-59 (airship)',
 'Zeppelin L-70 (airship)',
 'Zeppelin L-53 (airship)',
 'De Havilland DH-4',
 'FD Type Dirigible',
 'Caproni Ca.48',
 'Curtiss R-4LM',
 'De Havilland DH.4',
 'de Havilland DH-4',
 'Curtiss JN-4H',
 'Farman F-40',
 'Armstrong-Whitworth F-K-8',
 'Junkers JL-6',
 'Junkers F-13',
 'Salmson 2-A-2',
 'Breguet 14',
 'Handley Page HP-16',
 'Avro 504',
 'De Havilland DH-4\rDe Havilland DH-4',
 'Royal Airship Works ZR-2 (airship)',
 'Potez IX',
 'Bristol 28 Tou

In [105]:
def tipo_transporte(x):
    word_list=['Zeppel', 'Dirigible', 'boat', 'Boat', 'amphib', 'helicopter', 'copter', 'heli']
    a=0
    for i in word_list: 
        if i in x:
            a = 1
    if a == 1:
        return 'Otro'

In [106]:
data['tipo_avion'] = data['ac_type'].apply(tipo_transporte)

In [107]:
data.loc[data[data['tipo_avion'].isnull()].index,'tipo_avion'] = data['summary'].apply(tipo_transporte)

In [108]:
data['tipo_avion'].fillna('Avión',inplace=True)

In [109]:
data

,fecha,Ruta,OperadOR,flight_no,route,ac_type,registration,all_aboard,PASAJEROS A BORDO,crew_aboard,...,crew_fatalities,ground,summary,localizar,country,lat,lon,tipo_vuelo,tipo_avion,intencional
0,1908-09-17,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,2.0,1.0,1.0,...,0.0,0.0,"During a demonstration flight, a U.S. Army fly...",Virginia,United States,37.123224,-78.492772,Militar,Avión,0
1,1909-09-07,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,1.0,0.0,1.0,...,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...,France,France,46.603354,1.888334,Otro,Avión,0
2,1912-07-12,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,5.0,0.0,5.0,...,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,New Jersey,United States,40.075738,-74.404162,Militar,Otro,0
3,1913-08-06,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,1.0,0.0,1.0,...,1.0,0.0,The first fatal airplane accident in Canada oc...,Canada,Canada,61.066692,-107.991707,Otro,Avión,0
4,1913-09-09,North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,20.0,15.0,5.0,...,4.0,0.0,The airship flew into a thunderstorm and encou...,North Sea,Water,56.741234,2.891172,Militar,Otro,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2021-03-28,"Butte, Alaska",Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,6.0,5.0,1.0,...,1.0,0.0,The sightseeing helicopter crashed after missi...,Alaska,United States,64.445961,-149.680909,Otro,Otro,0
4996,2021-05-21,"Kaduna, Nigeria",Military - Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,11.0,7.0,4.0,...,4.0,0.0,"While on final approach, in poor weather condi...",Nigeria,Nigeria,9.600036,7.999972,Militar,Avión,0
4997,2021-06-10,"Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,14.0,12.0,2.0,...,1.0,0.0,The plane was carrying military personnel and ...,Myanmar,Myanmar,17.175050,95.999965,Militar,Avión,0
4998,2021-07-04,"Patikul, Sulu, Philippines",Military - Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,96.0,88.0,8.0,...,4.0,3.0,"While attempting to land at Jolo Airport, the ...",Philippines,Philippines,12.750349,122.731210,Militar,Avión,0


Columna: summary

In [78]:
# Esta columna la voy a usar para detectar si hubo intencionalidad en el accidente, buscando palabras como bomb o hijack o terrorist

In [110]:
def bomb(x):
    if 'bomb' in str(x):
        return 1
    elif 'hijack' in str(x):
        return 1
    elif 'terrorist' in str(x):
        return 1       
    else:
        return 0   

In [111]:
data['intencional']=0
data.loc[data[data['tipo_vuelo']=='Comercial'].index,'intencional'] = data['summary'].apply(lambda x: bomb(x))

In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fecha                   5000 non-null   datetime64[ns]
 1   Ruta                    5000 non-null   object        
 2   OperadOR                5000 non-null   object        
 3   flight_no               5000 non-null   object        
 4   route                   5000 non-null   object        
 5   ac_type                 5000 non-null   object        
 6   registration            5000 non-null   object        
 7   all_aboard              5000 non-null   float64       
 8   PASAJEROS A BORDO       5000 non-null   float64       
 9   crew_aboard             5000 non-null   float64       
 10  cantidad de fallecidos  5000 non-null   float64       
 11  passenger_fatalities    5000 non-null   float64       
 12  crew_fatalities         5000 non-null   float64 

In [113]:
# Preparo el dataset para exportar con los nombres de columna normalizados; y las columnas que me interesan

data2=pd.DataFrame()
data2[[ 'fecha',
        'operador',
        'todos_abordo',
        'pasajeros_abordo',
        'tripul_abordo',
        'tot_fallecidos',
        'pasajeros_fallecidos',
        'tripul_fallecidos',
        'ground',
        'summary',
        'tipo_vuelo',
        'tipo_avion',
        'intencional',
        'pais',
        'lat',
        'lon' 
        ]] = data[[
        'fecha',
        'OperadOR',
        'all_aboard',
        'PASAJEROS A BORDO',
        'crew_aboard',
        'cantidad de fallecidos',
        'passenger_fatalities',
        'crew_fatalities',
        'ground',
        'summary',
        'tipo_vuelo',
        'tipo_avion',
        'intencional',
        'country',
        'lat',
        'lon'
        ]]

In [114]:
# data.to_csv('output_preprocessing\datos_completos.csv', index=False) esta línea esxportaría los datos completos
data2.to_csv('output_preprocessing\datos_mysql.csv', index=False) #esta sólo los datos que se ingestarán en la BD.